In [2]:
import sys
sys.path.append("/home/jarlehti/projects/gradu")

In [3]:
import os
import jax
import jax.numpy as jnp
import numpy as np
import pandas as pd
import d3p
from src.napsu_mq.napsu_mq import NapsuMQModel, NapsuMQResult

In [4]:
CURRENT_FOLDER = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
DATASETS_FOLDER = os.path.join(CURRENT_FOLDER, "data", "datasets")
MODELS = os.path.join(CURRENT_FOLDER, "models")

In [5]:
rng = jax.random.PRNGKey(12903863272)

In [6]:
dataset = pd.read_csv(os.path.join(DATASETS_FOLDER, "binary4d.csv"))
n, d = dataset.shape
print(n)
print(d)

100000
4


In [7]:
column_feature_set = [
    ('A', 'D'), 
    ('B', 'D'), 
    ('C', 'D'), 
]

model = NapsuMQModel()
result = model.fit(
    data=dataset,
    dataset_name="binary4d",
    rng=rng,
    epsilon=50,
    delta=(n ** (-2)),
    column_feature_set=column_feature_set,
    use_laplace_approximation=True
)

No experiment_id found: <ContextVar name='experiment_id' at 0x7f5248e62d60>
Setting experiment_id to E2QYJY18
Recording:  Query selection


/home/jarlehti/projects/gradu/src/napsu_mq/mst.py:130: RuntimeWarning: divide by zero encountered in double_scalars
  epsilon = np.sqrt(8 * rho / (r - 1))


Recording:  Calculating full marginal query
Recording:  Calculating canonical query set
Calculating canonical queries


  0%|                                                                                                                | 0/8 [00:00<?, ?it/s]
4it [00:00, 7090.96it/s]

2it [00:00, 8405.42it/s]

4it [00:00, 13865.47it/s]

4it [00:00, 15210.53it/s]

2it [00:00, 12318.07it/s]

2it [00:00, 8507.72it/s]

2it [00:00, 7788.87it/s]
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 60494.77it/s]


Calculating new queries


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 11949.58it/s]


Canonical queries: 7
Domain d: 4
Nodes after elimination: {('C', 'D'), ('D',), ('B', 'D'), ('A', 'D')}
Edges after elimination: {(('A', 'D'), ('D',)): {'D'}, (('C', 'D'), ('D',)): {'D'}, (('B', 'D'), ('D',)): {'D'}}
Nodes after removing: {('C', 'D'), ('B', 'D'), ('A', 'D')}
Edges after removing: {(('C', 'D'), ('B', 'D')): {'D'}, (('C', 'D'), ('A', 'D')): {'D'}}
Suff stat d: 7
Lambda d: 7
[TreeNode(variables=('B', 'D'), parent=('C', 'D'), children=[]), TreeNode(variables=('A', 'D'), parent=('C', 'D'), children=[])]
1
Junction tree width: 2
Recording:  Laplace approximation
Attempting Laplace approximation, 0th try
Attempting Laplace approximation, 1th try
Attempting Laplace approximation, 2th try
Attempting Laplace approximation, 3th try
Attempting Laplace approximation, 4th try
Attempting Laplace approximation, 5th try
Attempting Laplace approximation, 6th try
Attempting Laplace approximation, 7th try
Attempting Laplace approximation, 8th try
Attempting Laplace approximation, 9th try


ConvergenceException: Minimize function failed to converge with 10 retries

In [7]:
result_path = os.path.join(MODELS, "napsu_testing_binary4d_all_queries_500e_NUTS_LA.dill")
result.store(result_path)

In [11]:
napsu_result_read_file = open(os.path.join(MODELS, "napsu_testing_binary4d_all_queries_500e_NUTS_LA.dill"), "rb")
loaded_result = NapsuMQResult.load(napsu_result_read_file)
sampling_rng = jax.random.PRNGKey(234513465234)
datasets = loaded_result.generate_extended(rng=sampling_rng, num_data_per_parameter_sample=2000, num_parameter_samples=5)

Generating data with 2000 points and 5 datasets


In [12]:
datasets

[      A  B  C  D
 0     0  1  0  1
 1     0  1  1  1
 2     1  0  0  1
 3     1  1  1  1
 4     0  1  1  1
 ...  .. .. .. ..
 1995  0  1  0  1
 1996  1  1  1  1
 1997  0  0  0  0
 1998  1  1  1  1
 1999  1  0  0  1
 
 [2000 rows x 4 columns],
       A  B  C  D
 0     1  0  0  1
 1     0  1  1  1
 2     0  1  0  0
 3     0  1  0  1
 4     0  0  1  0
 ...  .. .. .. ..
 1995  0  0  1  1
 1996  1  0  0  1
 1997  0  0  1  1
 1998  1  1  0  0
 1999  1  0  1  1
 
 [2000 rows x 4 columns],
       A  B  C  D
 0     1  1  1  1
 1     0  0  0  1
 2     0  1  1  1
 3     1  1  0  1
 4     1  1  0  1
 ...  .. .. .. ..
 1995  1  1  0  1
 1996  1  1  1  1
 1997  1  0  1  1
 1998  0  0  1  1
 1999  1  1  0  1
 
 [2000 rows x 4 columns],
       A  B  C  D
 0     1  1  0  0
 1     1  1  0  1
 2     1  0  0  1
 3     1  0  0  1
 4     0  0  1  1
 ...  .. .. .. ..
 1995  0  1  1  1
 1996  0  0  0  0
 1997  1  1  1  1
 1998  0  0  1  1
 1999  0  0  0  1
 
 [2000 rows x 4 columns],
       A  B  C  D
 0    